In [8]:
import pandas as pd
import numpy as np
import random
import uuid
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Step 1: Load dataset (you can load your dataset here)
# For the sake of example, we will create a dummy dataset
data = {
    'Vehicle_ID': [f'EV{random.randint(1000, 9999)}' for _ in range(100)],
    'Energy_Consumed_kWh': np.random.uniform(10, 100, 100),
    'Charging_Duration': np.random.uniform(1, 6, 100),
    'Cost': np.random.uniform(20, 200, 100)
}

df = pd.DataFrame(data)

# Step 2: Preprocess the dataset
# We'll drop the 'Vehicle_ID' as it is not relevant for prediction
X = df[['Energy_Consumed_kWh', 'Charging_Duration']]
y = df['Cost']

# Split data into train and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define models
models = {
    "LinearRegression": LinearRegression(),
    "RandomForestRegressor": RandomForestRegressor(random_state=42),
    "SVR": SVR()
}

# To store the performance of each model
model_performance = {}

# Step 4: Train and evaluate multiple models
for model_name, model in models.items():
    # Create a pipeline to scale the data (important for SVR)
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Normalize data for SVR and others
        (model_name, model)  # The model
    ])
    
    # Cross-validation to evaluate model performance
    scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    rmse_scores = np.sqrt(-scores)
    
    # Store the model's mean RMSE
    model_performance[model_name] = rmse_scores.mean()
    print(f"{model_name}: RMSE = {rmse_scores.mean():.4f} ± {rmse_scores.std():.4f}")

# Step 5: Choose the best model based on lowest RMSE
best_model_name = min(model_performance, key=model_performance.get)
best_model = models[best_model_name]


# Train the best model on the entire training dataset
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    (best_model_name, best_model)
])
best_pipeline.fit(X_train, y_train)

# Step 6: Evaluate the best model on the test set
y_pred = best_pipeline.predict(X_test)

# Calculate evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Best model: {best_model_name}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")


LinearRegression: RMSE = 50.2552 ± 2.5974
RandomForestRegressor: RMSE = 53.2121 ± 2.9644
SVR: RMSE = 51.0371 ± 2.8581
Best model: LinearRegression
Test RMSE: 51.6655
Test R²: -0.0082


In [9]:
df.to_csv("ev_charging.csv")

In [5]:
df.head()

,Vehicle_ID,Energy_Consumed_kWh,Charging_Duration,Cost
0,EV6618,77.511668,1.268858,167.242388
1,EV3125,81.943443,1.423668,68.128192
2,EV1133,88.769062,5.095885,66.966112
3,EV1579,81.529063,5.983158,140.612820
4,EV1303,54.039709,2.992133,87.602046


In [10]:
import csv

# Define the new data to add to the CSV
new_data = [
    {'Vehicle_ID': 'EV1234', 'Energy_Consumed_kWh': 55.0, 'Charging_Duration': 3.5, 'Cost': 19.25},
    {'Vehicle_ID': 'EV5678', 'Energy_Consumed_kWh': 40.0, 'Charging_Duration': 2.8, 'Cost': 15.00},
    {'Vehicle_ID': 'EV9012', 'Energy_Consumed_kWh': 65.0, 'Charging_Duration': 4.0, 'Cost': 23.00}
]

# Path to your CSV file
csv_file = 'ev_charging.csv'

# Write new data to CSV
with open(csv_file, mode='a', newline='') as file:
    fieldnames = ['Vehicle_ID', 'Energy_Consumed_kWh', 'Charging_Duration', 'Cost']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Append each row in the new data list
    for row in new_data:
        writer.writerow(row)

print(f"Data has been appended to {csv_file}.")


Data has been appended to ev_charging.csv.


In [7]:
df.head()

,Vehicle_ID,Energy_Consumed_kWh,Charging_Duration,Cost
0,EV6618,77.511668,1.268858,167.242388
1,EV3125,81.943443,1.423668,68.128192
2,EV1133,88.769062,5.095885,66.966112
3,EV1579,81.529063,5.983158,140.612820
4,EV1303,54.039709,2.992133,87.602046
